In [1]:
# Step 1: Creating and starting the SAMP HUB

from astropy.vo.samp import SAMPHubServer
hub = SAMPHubServer()
hub.start()

INFO:astropy:Hub set to run with Web Profile support enabled.


INFO: Hub set to run with Web Profile support enabled. [astropy.vo.samp.hub]
INFO

INFO:astropy:Hub started


: Hub started [astropy.vo.samp.hub]


In [2]:
# Step 2: Initializing the receiver

import urllib
#from astropy.table import Table
from astropy.vo.samp import SAMPIntegratedClient

client = SAMPIntegratedClient()
client.connect()

class Receiver(object):
    def __init__(self, client):
        self.client = client
        self.received = False
    def receive_call(self, private_key, sender_id, msg_id, mtype, params, extra):
        self.params = params
        self.received = True
        self.client.reply(msg_id, {"samp.status": "samp.ok", "samp.result": {}})
    def receive_notification(self, private_key, sender_id, mtype, params, extra):
        self.params = params
        self.received = True

r = Receiver(client)

client.bind_receive_call("table.load.votable", r.receive_call)
client.bind_receive_notification("table.load.votable", r.receive_notification)

client.bind_receive_call("table.load.cdf", r.receive_call)
client.bind_receive_notification("table.load.cdf", r.receive_notification)

client.bind_receive_call("table.load.fits", r.receive_call)
client.bind_receive_notification("table.load.fits", r.receive_notification)

In [3]:
# Step 3: Waiting for SAMP message

while (r.received == False):
    pass

>>> yes
A Web application which declares to be

Name: Vespa
Origin: http://vespa.obspm.fr

is requesting to be registered with the SAMP Hub.
Pay attention that if you permit its registration, such
application will acquire all current user privileges, like
file read/write.

Do you give your consent? [yes|no]


In [ ]:
# Step 4: Download files

import os

if len(r.params) == 2:
    # This is the case of the full dataset
    instrument=r.params['name']
    print 'FUNCTION NOT ALREADY IMPLEMENTED'
else:
    # This is the case of the single image cube
    
    # First we check if the url links a CAL or a GEO file
    # Then we create url for CAL and GEO files and download them
    
    if os.path.splitext(r.params['url'])[1] == '.CAL':
        cal_url=r.params['url']
        split_url=cal_url.split('/')
        geo_url=[]
        for i in range(len(split_url)):
            if split_url[i]=='CALIBRATED': 
                geo_url.append('GEOMETRY')
            else:
                geo_url.append(split_url[i])
        cal_name=os.path.basename(cal_url)
        geo_url='/'.join(geo_url)
        geo_url=geo_url[0:len(geo_url)-3]+'GEO'
        geo_name=os.path.basename(geo_url)
    else:
        geo_url=r.params['url']
        split_url=geo_url.split('/')
        cal_url=[]
        for i in range(len(split_url)):
            if split_url[i]=='GEOMETRY': 
                cal_url.append('CALIBRATED')
            else:
                cal_url.append(split_url[i])
        geo_name=os.path.basename(geo_url)
        cal_url='/'.join(cal_url)
        cal_url=cal_url[0:len(cal_url)-3]+'CAL'
        cal_name=os.path.basename(cal_url)
        
    print 'Files to be donwloaded\n'
    print 'CAL file: '+cal_url+'\n  --> to '+os.path.join('f:',cal_name)
    print 'GEO file: '+geo_url+'\n  --> to '+os.path.join('f:',geo_name)
        

In [4]:
    # Download CAL and GEO files
    
    calfile = urllib.URLopener()
    calfile.retrieve(cal_url,os.path.join('f:',cal_name))
    calfile.close()
    
    geofile = urllib.URLopener()
    geofile.retrieve(geo_url,os.path.join('f:',geo_name))
    geofile.close()

Files to be donwloaded

CAL file: ftp://psa.esac.esa.int/pub/mirror/VENUS-EXPRESS/VIRTIS/VEX-V-VIRTIS-2-3-V3.0/DATA/MTP019/VIR0520/CALIBRATED/VV0520_00.CAL
 to f:VV0520_00.CAL
GEO file: ftp://psa.esac.esa.int/pub/mirror/VENUS-EXPRESS/VIRTIS/VEX-V-VIRTIS-2-3-V3.0/DATA/MTP019/VIR0520/GEOMETRY/VV0520_00.GEO
 to f:VV0520_00.GEO
